# EJERCICIO NLP
## Extracción de N.E.R desde un artículo

Vamos a aplicar lo visto hasta ahora sobre NLP para realizar una extracción de NER sobre un artículo de prensa.

1.- Carga de librerías

In [1]:
import spacy
from spacy import displacy
from collections import Counter
import es_core_news_lg

nlp = spacy.load('es_core_news_lg')

2. Detección de entidades

In [ ]:
# pip install html5lib
# pip install bs4
# pip install requests

# html5lib se usa para limpiar el html y bs4 para parsear el html y obtener el texto
# requests se usa para hacer la petición a la página web
# re es para hacer la limpieza de texto

from bs4 import BeautifulSoup
import requests
import re
def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))
veinte_min = url_to_string('https://www.20minutos.es/internacional/trump-hablara-martes-putin-desbloquear-iniciativa-paz-eeuu-ucrania-5691587/')
article = nlp(veinte_min)
len(article.ents)

100

Se detectan en este artículo 100 entidades, repartidas como sigue:

In [4]:
labels = [x.label_ for x in article.ents]
Counter(labels)

Counter({'LOC': 36, 'MISC': 28, 'PER': 26, 'ORG': 10})

Las 3 entidades más repetidas son:

In [5]:
items = [x.text for x in article.ents]
Counter(items).most_common(3)

[('Ucrania', 11), ('Putin', 8), ('Trump', 7)]

Análisis de una frase del artículo:

In [11]:
sentences = [x for x in article.sents]
print(sentences[19])

"Parte de estas garantías debería de ser el estatus neutral de Ucrania y la negativa de los países de la OTAN a aceptarla como miembro de la Alianza.


Usamos displaCy para resaltar las entidades:

In [22]:
displacy.render(nlp(str(sentences[19])), jupyter=True, style='ent')

In [17]:
displacy.render(nlp(str(sentences[19])), style='dep', jupyter = True, options = {'distance': 100})

In [18]:
[(x.orth_,x.pos_, x.lemma_) for x in [y 
                                      for y
                                      in nlp(str(sentences[19])) 
                                      if not y.is_stop and y.pos_ != 'PUNCT']]

[('garantías', 'NOUN', 'garantía'),
 ('debería', 'AUX', 'deber'),
 ('estatus', 'NOUN', 'estatus'),
 ('neutral', 'ADJ', 'neutral'),
 ('Ucrania', 'PROPN', 'Ucrania'),
 ('negativa', 'NOUN', 'negativa'),
 ('países', 'NOUN', 'país'),
 ('OTAN', 'PROPN', 'OTAN'),
 ('aceptarla', 'VERB', 'aceptar él'),
 ('miembro', 'NOUN', 'miembro'),
 ('Alianza', 'PROPN', 'Alianza')]

In [19]:
dict([(str(x), x.label_) for x in nlp(str(sentences[19])).ents])

{'Ucrania': 'LOC', 'OTAN': 'ORG', 'Alianza': 'MISC'}

In [20]:
print([(x, x.ent_iob_, x.ent_type_) for x in sentences[19]])

[("Parte, 'O', ''), (de, 'O', ''), (estas, 'O', ''), (garantías, 'O', ''), (debería, 'O', ''), (de, 'O', ''), (ser, 'O', ''), (el, 'O', ''), (estatus, 'O', ''), (neutral, 'O', ''), (de, 'O', ''), (Ucrania, 'B', 'LOC'), (y, 'O', ''), (la, 'O', ''), (negativa, 'O', ''), (de, 'O', ''), (los, 'O', ''), (países, 'O', ''), (de, 'O', ''), (la, 'O', ''), (OTAN, 'B', 'ORG'), (a, 'O', ''), (aceptarla, 'O', ''), (como, 'O', ''), (miembro, 'O', ''), (de, 'O', ''), (la, 'O', ''), (Alianza, 'B', 'MISC'), (., 'O', '')]


3. Resaltado de todo el artículo.

In [21]:
displacy.render(article, jupyter=True, style='ent')